<a href="https://colab.research.google.com/github/paulosalgado/OpenBankingBR_Participantes/blob/main/Open_Banking_Participantes_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import urllib.request as request_lib
import json

In [2]:
# CONFIG
participants_url = 'https://data.directory.openbankingbrasil.org.br/participants'

participants_request = request_lib.Request(participants_url)
participants_response = request_lib.urlopen(participants_request)
participants = json.loads(participants_response.read().decode('utf-8'))

print(participants)

[{'Status': 'Active', 'OrgDomainRoleClaims': [{'Status': 'Inactive', 'AuthorisationDomain': 'Open Banking', 'Role': 'PAGTO', 'RegistrationId': '07853842', 'Authorisations': []}, {'Status': 'Active', 'AuthorisationDomain': 'Open Banking', 'Role': 'CONTA', 'RegistrationId': '07853842', 'Authorisations': []}, {'Status': 'Inactive', 'AuthorisationDomain': 'Open Banking', 'Role': 'DADOS', 'RegistrationId': '07853842', 'Authorisations': []}], 'AuthorisationServers': [{'PayloadSigningCertLocationUri': 'https://api.sulcrediouro.com.br/open-banking/naoseaplica', 'ParentAuthorisationServerId': None, 'OpenIDDiscoveryDocument': 'https://api.sulcrediouro.com.br/open-banking/naoseaplica', 'CustomerFriendlyName': 'OURO', 'CustomerFriendlyDescription': 'Open Banking por CashWay', 'TermsOfServiceUri': 'https://api.sulcrediouro.com.br/open-banking/naoseaplica', 'ApiResources': [{'ApiFamilyType': 'admin', 'ApiVersion': 1, 'FamilyComplete': False, 'CertificationExpirationDate': None, 'ApiResourceId': '675

In [3]:
# CONFIG
search_registered_name = 'itau'.lower()

organisations = list(filter(lambda participant: search_registered_name in participant['RegisteredName'].lower(), participants))
print('Found {} organization(s)\n'.format(len(organisations)))

for organization in organisations: 
  print(' - {}\n   - OrganisationId: {}\n'.format(organization['RegisteredName'], organization['OrganisationId']))

Found 4 organization(s)

 - BANCO ITAUCARD S.A.
   - OrganisationId: b6b68aba-7b89-5981-8679-28a4244df514

 - ITAU CORRETORA DE VALORES S.A.
   - OrganisationId: a773281f-9897-5e23-b422-ec72735f4f12

 - BANCO ITAUBANK S.A.
   - OrganisationId: a3280c20-efe2-5634-907f-6ef2a42d2a7d

 - ITAUVEST DISTRIBUIDORA DE TITULOS E VALORES MOBILIARIOS S.A.
   - OrganisationId: 67fdb0c2-bda1-52d8-a7dc-46ad1548dd76



In [4]:
# CONFIG
organisation_id = 'b6b68aba-7b89-5981-8679-28a4244df514'

authorisation_servers = list(filter(lambda organisation: organisation_id in organisation['OrganisationId'], organisations))[0]['AuthorisationServers']
print('Found {} authorisation server(s)\n'.format(len(authorisation_servers)))

for authorisation_server in authorisation_servers: 

  api_resources = authorisation_server['ApiResources']
  api_family_types = list(map(lambda resource: resource['ApiFamilyType'], api_resources))

  print(' - {}\n   - AuthorisationServerId: {}\n   - Well Known: {}\n   - API Resources: {} found - {}\n'.format(
      authorisation_server['CustomerFriendlyName'], 
      authorisation_server['AuthorisationServerId'], 
      authorisation_server['OpenIDDiscoveryDocument'], 
      len(api_resources), ', '.join(api_family_types)))

Found 5 authorisation server(s)

 - Iti
   - AuthorisationServerId: 8197a3c9-11ad-4cfd-9bbf-4d6ad5d51f83
   - Well Known: https://id.iti.opf.itau.com.br/.well-known/openid-configuration
   - API Resources: 14 found - resources, products-services, discovery, credit-cards-accounts, payments-consents, accounts, channels, payments-pix, consents, customers-business, loans, admin, financings, customers-personal

 - Itaú
   - AuthorisationServerId: 3944d195-e295-488f-9782-05fcfcc64d33
   - Well Known: https://id.opf.itau.com.br/.well-known/openid-configuration
   - API Resources: 12 found - consents, payments-pix, accounts, invoice-financings, customers-business, credit-cards-accounts, financings, unarranged-accounts-overdraft, resources, payments-consents, customers-personal, loans

 - Itaú BBA
   - AuthorisationServerId: 2379d0f6-f238-4855-b914-a82265922691
   - Well Known: https://id.itaubba.opf.itau.com.br/.well-known/openid-configuration
   - API Resources: 9 found - financings, credit-c

In [5]:
# CONFIG
authorisation_server_id = '3944d195-e295-488f-9782-05fcfcc64d33'

authorisation_server = list(filter(lambda authorisation_server: authorisation_server_id in authorisation_server['AuthorisationServerId'], authorisation_servers))[0]

well_known_url = authorisation_server['OpenIDDiscoveryDocument']
well_known_request = request_lib.Request(well_known_url)
well_known_response = request_lib.urlopen(well_known_request)
well_known = json.loads(well_known_response.read().decode('utf-8'))

print(json.dumps(well_known, indent=3))

{
   "issuer": "https://id.opf.itau.com.br",
   "pushed_authorization_request_endpoint": "https://oauth2.opf.itau/as/par.oauth2",
   "authorization_endpoint": "https://id.opf.itau.com.br/authorize",
   "token_endpoint": "https://oauth2.opf.itau/as/token.oauth2",
   "introspection_endpoint": "https://oauth2.opf.itau/introspect",
   "revocation_endpoint": "https://oauth2.opf.itau/revoke",
   "registration_endpoint": "https://oauth2.opf.itau/register",
   "userinfo_endpoint": "https://oauth2.opf.itau/userinfo",
   "jwks_uri": "https://id.opf.itau.com.br/JWKS",
   "scopes_supported": [
      "openid",
      "consents",
      "consent:*",
      "resources",
      "customers",
      "accounts",
      "credit-cards-accounts",
      "loans",
      "financings",
      "invoice-financings",
      "unarranged-accounts-overdraft"
   ],
   "claims_supported": [
      "acr",
      "consentId",
      "iss",
      "pi.sri",
      "sub"
   ],
   "require_pushed_authorization_requests": true,
   "code_c